In [3]:
import json
import firebase_admin
from firebase_admin import credentials, db
from pathlib import Path

private_keys = json.load(open("credentials.json"))
FIRESTORE_URL = private_keys["FIRESTORE_URL"] 

cred_file_path = Path("./firebase_key.json")
cred = credentials.Certificate(cred_file_path)

firebase_admin.initialize_app(
    cred, {"databaseURL": FIRESTORE_URL}
)
print(FIRESTORE_URL)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [4]:
ref = db.reference("/lettres")
print(ref.get())

[{'Department_Justification': "The letter discusses matters related to monetary policy and exchange controls, which fall under the remit of the HM Treasury as the government's economic and finance ministry.", 'Departments': 'HM Treasury', 'date': '1979-11-15', 'document-type': 'Letter', 'id': 0, 'image_url': 'https://storage.googleapis.com/bucket-images-correspondance/PREM_19-34_0001_-39.jpg', 'recipient': 'Chancellor', 'sender': 'C J RILEY', 'source': 'PREM 19-34/0001/-39.jpg', 'subject': 'Speech to the Institute of Bankers', 'text': "SECRET\n\nCHANCELLOR\n\nTreasurable\n(1) P.J Ledl\n(2) The Ptiley Hr.\n\nR. secroo:\n\ncc Chief Secretary\nFinancial Secretary\nSir D Mass\nSir L Airey\nSir F Atkinson\nSir K Couzens\nMr Barrett\nMr Littler\nMr Brideman\nMr Hancock\nMr Middleton\nMr Unwin\nMr P G Davies\nMr Pottrill\nMr Gill\nMrs Gilmore\nMr Hodges\nMrs Lomax\nMr Folger\nMr Williams\nMr Macrae\nMr Ridley\nMr Cardona\nMr Cropper\n\nMr Fforde\nChief Cashier\nMr Goodhart\n\nSPEECH TO THE IN

In [6]:
import pandas as pd
lettres_csv = Path("./data/single-page-letters.csv")
lettres_df = pd.read_csv(lettres_csv)
# Replace nans in dataset with "N/A"
lettres_df = lettres_df.fillna("N/A")

# Convert to dict
lettres_dict = lettres_df.to_dict(orient="records")
# Add an ID column
for i, lettre in enumerate(lettres_dict):
    lettre["id"] = i

#lettres_dict[0]
# Add the lettres to firebase

# Check that the dict is json serialisable
#for lettre in lettres_dict:
#    lid = lettre["id"]
#    print(lid, lettre)
#    ref.child(str(lid)).set(lettre)

print("Done")

Done


In [26]:
# Fetch the lettres from the db
lettres = ref.get()
print(type(ref.get()[0]))

<class 'dict'>


In [28]:
lettres[0]["source"]

'PREM 19-34/0001/-39.jpg'

In [33]:
# Upload images to GCS
from google.cloud import storage

BUCKET_NAME = "bucket-images-correspondance"
CREDENTIALS_FILE = Path("./gcs_credentials.json")

def upload_to_gcs(bucket_name, source_file_path, destination_blob_name, credentials_file):
    # Initialize the Google Cloud Storage client with the credentials
    storage_client = storage.Client.from_service_account_json(credentials_file)

    # Get the target bucket
    bucket = storage_client.bucket(bucket_name)

    # Upload the file to the bucket
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)

    print(f"File {source_file_path} uploaded to gs://{bucket_name}/{destination_blob_name}")

i = 0
for lettre in lettres_dict:
    i += 1
    lettre_id = lettre["id"]
    source_path = Path(f"./heavy_data/PREM Historical records/{lettre['source']}")
    destination_path = lettre["source"].replace(" ", "_").replace("/", "_")
    upload_to_gcs(BUCKET_NAME, source_path, destination_path, CREDENTIALS_FILE)
    print(f"Uploaded {i} images to GCS")

print("Done uploading images to GCS")

File heavy_data/PREM Historical records/PREM 19-34/0001/-39.jpg uploaded to gs://bucket-images-correspondance/PREM_19-34_0001_-39.jpg
Uploaded 1 images to GCS
File heavy_data/PREM Historical records/PREM 19-34/0001/-109.jpg uploaded to gs://bucket-images-correspondance/PREM_19-34_0001_-109.jpg
Uploaded 2 images to GCS
File heavy_data/PREM Historical records/PREM 19-34/0001/-140.jpg uploaded to gs://bucket-images-correspondance/PREM_19-34_0001_-140.jpg
Uploaded 3 images to GCS
File heavy_data/PREM Historical records/PREM 19-34/0001/-59.jpg uploaded to gs://bucket-images-correspondance/PREM_19-34_0001_-59.jpg
Uploaded 4 images to GCS
File heavy_data/PREM Historical records/PREM 19-34/0001/-192.jpg uploaded to gs://bucket-images-correspondance/PREM_19-34_0001_-192.jpg
Uploaded 5 images to GCS
File heavy_data/PREM Historical records/PREM 19-34/0001/-193.jpg uploaded to gs://bucket-images-correspondance/PREM_19-34_0001_-193.jpg
Uploaded 6 images to GCS
File heavy_data/PREM Historical record

In [35]:
def get_image_url_from_gcs(bucket_name, blob_name, credentials_file):
    # Initialize the Google Cloud Storage client with the credentials
    storage_client = storage.Client.from_service_account_json(credentials_file)

    # Get the target bucket
    bucket = storage_client.bucket(bucket_name)

    # Get the blob URL
    blob = bucket.blob(blob_name)
    return blob.public_url

# Get the URL of the first image
get_image_url_from_gcs(BUCKET_NAME, "PREM_19-175_0001_-175.jpg", CREDENTIALS_FILE)

'https://storage.googleapis.com/bucket-images-correspondance/PREM_19-175_0001_-175.jpg'

In [36]:
def add_image_url_to_firebase(ref, lettre_id, image_url):
    ref.child(str(lettre_id)).update({"image_url": image_url})
    
# Add the image URLs to the firebase
for lettre in lettres_dict:
    lettre_id = lettre["id"]
    image_url = get_image_url_from_gcs(BUCKET_NAME, lettre["source"].replace(" ", "_").replace("/", "_"), CREDENTIALS_FILE)
    add_image_url_to_firebase(ref, lettre_id, image_url)
    print(f"Added image URL for lettre {lettre_id}")

Added image URL for lettre 0
Added image URL for lettre 1
Added image URL for lettre 2
Added image URL for lettre 3
Added image URL for lettre 4
Added image URL for lettre 5
Added image URL for lettre 6
Added image URL for lettre 7
Added image URL for lettre 8
Added image URL for lettre 9
Added image URL for lettre 10
Added image URL for lettre 11
Added image URL for lettre 12
Added image URL for lettre 13
Added image URL for lettre 14
Added image URL for lettre 15
Added image URL for lettre 16
Added image URL for lettre 17
Added image URL for lettre 18
Added image URL for lettre 19
Added image URL for lettre 20
Added image URL for lettre 21
Added image URL for lettre 22
Added image URL for lettre 23
Added image URL for lettre 24
Added image URL for lettre 25
Added image URL for lettre 26
Added image URL for lettre 27
Added image URL for lettre 28
Added image URL for lettre 29
Added image URL for lettre 30
Added image URL for lettre 31
Added image URL for lettre 32
Added image URL for 

In [7]:
i = 0
for lettre in lettres_dict:
    i += 1
    lettre_id = lettre["id"]
    ref.child(str(lettre_id)).update({"summary": lettre["Summary"]})
    print(f"Uploaded {i} images to GCS")

Uploaded 1 images to GCS
Uploaded 2 images to GCS
Uploaded 3 images to GCS
Uploaded 4 images to GCS
Uploaded 5 images to GCS
Uploaded 6 images to GCS
Uploaded 7 images to GCS
Uploaded 8 images to GCS
Uploaded 9 images to GCS
Uploaded 10 images to GCS
Uploaded 11 images to GCS
Uploaded 12 images to GCS
Uploaded 13 images to GCS
Uploaded 14 images to GCS
Uploaded 15 images to GCS
Uploaded 16 images to GCS
Uploaded 17 images to GCS
Uploaded 18 images to GCS
Uploaded 19 images to GCS
Uploaded 20 images to GCS
Uploaded 21 images to GCS
Uploaded 22 images to GCS
Uploaded 23 images to GCS
Uploaded 24 images to GCS
Uploaded 25 images to GCS
Uploaded 26 images to GCS
Uploaded 27 images to GCS
Uploaded 28 images to GCS
Uploaded 29 images to GCS
Uploaded 30 images to GCS
Uploaded 31 images to GCS
Uploaded 32 images to GCS
Uploaded 33 images to GCS
Uploaded 34 images to GCS
Uploaded 35 images to GCS
Uploaded 36 images to GCS
Uploaded 37 images to GCS
Uploaded 38 images to GCS
Uploaded 39 images to